In [1]:
import param
import panel as pn
import pandas as pd
import time
import requests

from io import StringIO


import plotly.express as px

pn.extension('plotly', 'tabulator', sizing_mode="stretch_width")

In [2]:
class AudioDashboard(param.Parameterized):
    data = param.Dict()
  
    file_input = param.Parameter()
    
    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.TextInput(), **params)
        self.transcript = pn.pane.Str(max_width = 600, height_policy = "fit")
        self.download = pn.widgets.FileDownload(name="Download transcript", filename="transcript.txt", callback=self._download_callback, button_type="primary")

    @pn.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        print("parse input ")
        print(self.file_input)
        value = self.file_input.value
        if value:
            print("in parse")
            self.data = self.get_and_analyze_transcript()
            print(self.data)
            
        else:
            print("error")

    @pn.depends('data', watch=True)
    def get_transcript(self):
        print("get transcript ")
        self.transcript.object = self.data["text"]
        
        
    def _download_callback(self):
        print("download")
        if self.data is not None:
            buffer = StringIO()
            buffer.write(self.data["text"])
            buffer.seek(0)
            print("return buffer")

            return buffer  
        else:
            return
    
    def poll(self):
        polling_endpoint = transcript_endpoint + '/' + self.transcript_id
        polling_response = requests.get(polling_endpoint, headers=self.headers)
        self.data = polling_response.json()
        
    def get_and_analyze_transcript(self):
        audio_url = self.file_input.value
                
        if audio_url:
            print("yessss")
            API_KEY = "746d75f8f2ba44d99ae1254fa3da07f3"

            endpoint = "https://api.assemblyai.com/v2/transcript"
            audio_url_json = {
                "audio_url": audio_url,
                "sentiment_analysis": True,
                "auto_highlights": True,
                "iab_categories": True,
                "auto_chapters": True
            }
            headers = {
                "authorization": API_KEY,
                "content-type": "application/json"
            }
            transcript_response = requests.post(endpoint, json=audio_url_json, headers=headers)

            result_endpoint = endpoint + '/' + transcript_response.json()["id"]
            headers_auth = {
                "authorization": API_KEY,
            }
            response = requests.get(result_endpoint, headers=headers_auth)
            while response.json()['status'] != "completed":
                response = requests.get(result_endpoint, headers=headers_auth)
                time.sleep(3)
                
            return response.json()

        else:
            print("no")
            return
        
    def view(self):
        return pn.Column(
            "## Transcript",
            self.transcript,
        )
    
audio_app = AudioDashboard()

audio_app_view = audio_app.view()
audio_app_view
        

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Str(None, height_policy='fit', max_width=600, sizing_mode='stretch_width')

In [3]:
description = """
This is my app :)
"""

component = pn.Column(
    description,
    audio_app_view,
    sizing_mode='stretch_both'
)
component

template = pn.template.FastListTemplate(
    title='Audio Content Explorer', 
    sidebar=[pn.pane.Markdown("### Input a link:"), 
             audio_app.file_input, 
             audio_app.download],
    main=[component],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

template.servable()


parse input 
TextInput(sizing_mode='stretch_width', value='https://github.com/emily8..., value_input='https://github.com/emily8...) 
in parse 
yessss 
get transcript 
{'id': 'rg9s74tk7g-133c-4ce8-9c74-5df034abfd6c', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'completed', 'audio_url': 'https://github.com/emily876/sentiment-analysis-of-my-fav-song/blob/main/Why%20Don%20t%20We%20%E2%80%92%20Big%20Plans%20Lyrics.mp3?raw=true', 'text': "Ain't got everything you want but got everything you need just take a chance take a chance only I got reobby ain't got everything you want but got everything you need baby give me one year yeah take my hand take a leap girl. You gotta wait and see where I take it from here that's just a beautiful thing about you I know that you do the same got everything you want but got everything you need to take a chance take a chance on me I got reopened. Baby, love me for who I am oh, my God. Hold our baby take a chance take a chance take a chance.", 'words': [{'text': "Ain't", 'start': 37512, 'end': 37707, 'confidence': 0.51982, 'speaker': None}, {'text': 'got', 'start': 37722, 'end': 37942, 'confidence': 0.99369, 'speaker': None}, {'text': 'everything', 'start': 37990, 'end': 38332, 'confidence': 0.99636, 'speaker': None}, {'text': 'you', 'start': 38410, 'end': 38647, 'confidence': 0.55, 'speaker': None}, {'text': 'want', 'start': 38680, 'end': 38962, 'confidence': 0.51919, 'speaker': None}, {'text': 'but', 'start': 39025, 'end': 39247, 'confidence': 0.88241, 'speaker': None}, {'text': 'got', 'start': 39280, 'end': 39517, 'confidence': 0.95529, 'speaker': None}, {'text': 'everything', 'start': 39565, 'end': 39952, 'confidence': 0.99963, 'speaker': None}, {'text': 'you', 'start': 40045, 'end': 40297, 'confidence': 0.93, 'speaker': None}, {'text': 'need', 'start': 40330, 'end': 40927, 'confidence': 0.98868, 'speaker': None}, {'text': 'just', 'start': 41095, 'end': 41422, 'confidence': 0.52259, 'speaker': None}, {'text': 'take', 'start': 41455, 'end': 41647, 'confidence': 0.96271, 'speaker': None}, {'text': 'a', 'start': 41680, 'end': 41872, 'confidence': 0.6, 'speaker': None}, {'text': 'chance', 'start': 41905, 'end': 42187, 'confidence': 0.99866, 'speaker': None}, {'text': 'take', 'start': 42250, 'end': 42472, 'confidence': 0.96934, 'speaker': None}, {'text': 'a', 'start': 42505, 'end': 42652, 'confidence': 0.85, 'speaker': None}, {'text': 'chance', 'start': 42670, 'end': 43027, 'confidence': 0.61752, 'speaker': None}, {'text': 'only', 'start': 43120, 'end': 43642, 'confidence': 0.75, 'speaker': None}, {'text': 'I', 'start': 43765, 'end': 44002, 'confidence': 0.84, 'speaker': None}, {'text': 'got', 'start': 44020, 'end': 44242, 'confidence': 0.57132, 'speaker': None}, {'text': 'reobby', 'start': 44290, 'end': 48062, 'confidence': 0.04982, 'speaker': None}, {'text': "ain't", 'start': 50137, 'end': 50457, 'confidence': 0.50153, 'speaker': None}, {'text': 'got', 'start': 50472, 'end': 50692, 'confidence': 0.89148, 'speaker': None}, {'text': 'everything', 'start': 50740, 'end': 51172, 'confidence': 0.85024, 'speaker': None}, {'text': 'you', 'start': 51280, 'end': 51502, 'confidence': 0.8, 'speaker': None}, {'text': 'want', 'start': 51520, 'end': 51787, 'confidence': 0.82534, 'speaker': None}, {'text': 'but', 'start': 51850, 'end': 52072, 'confidence': 0.38446, 'speaker': None}, {'text': 'got', 'start': 52105, 'end': 52342, 'confidence': 0.54674, 'speaker': None}, {'text': 'everything', 'start': 52390, 'end': 52777, 'confidence': 0.99511, 'speaker': None}, {'text': 'you', 'start': 52870, 'end': 53077, 'confidence': 0.69, 'speaker': None}, {'text': 'need', 'start': 53095, 'end': 53675, 'confidence': 0.74197, 'speaker': None}, {'text': 'baby', 'start': 73687, 'end': 74247, 'confidence': 0.55113, 'speaker': None}, {'text': 'give', 'start': 74292, 'end': 74497, 'confidence': 0.97505, 'speaker': None}, {'text': 'me', 'start': 74530, 'end': 74722, '

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [2218575174992] Column(sizing_mode='stretch_both')
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Str(None, height_policy='fit', max_width=600, sizing_mode='stretch_width')
    [2218885020208] Markdown(str, sizing_mode='stretch_width')
    [2218884973424] TextInput(sizing_mode='stretch_width')
    [2218884974576] FileDownload(button_type='primary', callback=<bound method AudioDashboa..., filename='transcript.txt', label='Download transcript.txt', name='Download transcript', sizing_mode='stretch_width')